In [1]:
### IMPORTAR DEPENDENCIAS ###
import streamlit as st
from PyPDF2 import PdfReader, PdfFileReader
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from typing import List
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from pydantic import BaseModel, Field
import dotenv
import os
import regex as re

import warnings
warnings.filterwarnings('ignore')

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

c:\anaconda\anaconda3\envs\RAG_simple\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [62]:
# carga variables de entorno de .env
dotenv.load_dotenv()

True

In [59]:
# extrae texto de PDF
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# extrae text chunks (pedazos de texto)
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['(VOLUME\s+\d+\s+-\s+([A-Za-z\-])*\s+-\s+Pag\s+\d+\s+)'],
        is_separator_regex=True,
        keep_separator=True,
        chunk_size=2048, 
        chunk_overlap=256,
        )
    chunk_OK = False
    ajuste = 0
    cont = 0
    print('Chunking: procesando texto...')
    while not chunk_OK:
        try:
            chunks = text_splitter.split_text(text[-(len(text)-ajuste):])
            print('Texto procesado correctamente')
            chunk_OK = True
        except:
            cont += 1
            print(f'Ajustando largo de texto. Iteración {cont}')
            ajuste += 50
        
    chunks = [chunk for chunk in chunks if len(chunk)>90]
    
    return chunks

# vector store chromadb
def get_vector_store(text_chunks):
    Chroma.from_texts(texts=text_chunks,
                          embedding=OpenAIEmbeddings(
                              model='text-embedding-3-large'), 
                          persist_directory="./chroma_db",
                          )
    
# vector store pinecone
def vector_store_pinecone(text_chunks):
    PineconeVectorStore.from_documents(
        documents=text_chunks, 
        embedding=OpenAIEmbeddings(), 
        index_name=os.getenv('PINECONE_INDEX_NAME'))

### Archivos individuales

In [4]:
os.getcwd()

'c:\\Users\\CarlosGorricho(ext)\\Documents\\Manuales_local\\Oben_chatbot_OpenAI'

In [6]:
file = 'C:\\Users\\CarlosGorricho(ext)\\Documents\Manuales_local\\Oben_chatbot_OpenAI\\transform_manuals\\VOLUME 02 - LineOperationManual_clean_output_new.txt'

In [7]:
raw_text = open(file, 'r').read()

In [8]:
type(raw_text)

str

In [9]:
len(raw_text)

370734

In [29]:
chunks = get_text_chunks(raw_text)
chunks

Chunking: procesando texto...
Texto procesado correctamente


['VOLUME 02 - LineOperationManual - Pag 1 \nVOLUME 02 - LineOperationManual - Pag 1 \nlOperating manualLine operation manualP.1932 SHB\n\n--------------------------------------------------------------------------------\nVOLUME 02 - LineOperationManual - Pag 2 \nVOLUME 02 - LineOperationManual - Pag 2 \nl--------------------------------------------------------------------------------\nVOLUME 02 - LineOperationManual - Pag 3 \nVOLUME 02 - LineOperationManual - Pag 3',
 'lImprintThis operating manual is part of the equipment supplied by Brückner MaschinenbauGmbH & Co. KG - hereinafter called Brückner - and describes the necessary pro-cedures for the proper use of the film production line and the associated machines.CopyrightNo part of this operating manual may be reproduced, stored in a retrieval system ortransmitted, in any form or by any means, electronic, mechanical or by recording orotherwise without the prior permission by Brückner. The copying, distribution andutilisation of this op

In [30]:
print(chunks[4])

l7.4Eliminating a fault............................................................................................................................. 747.5Safety action on the film casting unit in the event of a malfunction..................................................  778 Shutting down the film production line.................................................................................................. 798.1Carrying out a Cut Back ................................................................................................................... 798.2Shutting down the film casting unit and extrusion systems..............................................................  879 Operating the line..................................................................................................................................... 919.1Operating the drives......................................................................................................................... 919.1.1Sett

In [14]:
len(chunks)

235

In [31]:
for i, chunk in enumerate(chunks):
    print(f'{i}. Largo del chunk: {len(chunk)}')

0. Largo del chunk: 457
1. Largo del chunk: 2070
2. Largo del chunk: 379
3. Largo del chunk: 5779
4. Largo del chunk: 6100
5. Largo del chunk: 5862
6. Largo del chunk: 5524
7. Largo del chunk: 3173
8. Largo del chunk: 1652
9. Largo del chunk: 1732
10. Largo del chunk: 1991
11. Largo del chunk: 2161
12. Largo del chunk: 1900
13. Largo del chunk: 462
14. Largo del chunk: 2736
15. Largo del chunk: 1735
16. Largo del chunk: 1538
17. Largo del chunk: 1692
18. Largo del chunk: 1170
19. Largo del chunk: 2159
20. Largo del chunk: 1796
21. Largo del chunk: 1751
22. Largo del chunk: 2028
23. Largo del chunk: 1872
24. Largo del chunk: 387
25. Largo del chunk: 2152
26. Largo del chunk: 248
27. Largo del chunk: 1984
28. Largo del chunk: 1021
29. Largo del chunk: 2032
30. Largo del chunk: 1428
31. Largo del chunk: 1664
32. Largo del chunk: 1854
33. Largo del chunk: 1618
34. Largo del chunk: 1387
35. Largo del chunk: 2004
36. Largo del chunk: 1044
37. Largo del chunk: 1530
38. Largo del chunk: 1604
3

### Todos los archivos

In [16]:
os.getcwd()

'c:\\Users\\CarlosGorricho(ext)\\Documents\\Manuales_local\\Oben_chatbot_OpenAI'

In [32]:
from pathlib import Path
path = os.getcwd() + '\\transform_manuals\\'
txt_docs = [path + file for file in os.listdir(path) if 'VOLUME' in file]
txt_docs

['c:\\Users\\CarlosGorricho(ext)\\Documents\\Manuales_local\\Oben_chatbot_OpenAI\\transform_manuals\\VOLUME 00 - OverviewDocumentation_clean_output_new.txt',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\Manuales_local\\Oben_chatbot_OpenAI\\transform_manuals\\VOLUME 01 - OperatingManual-LineOverview_clean_output_new.txt',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\Manuales_local\\Oben_chatbot_OpenAI\\transform_manuals\\VOLUME 02 - LineOperationManual_clean_output_new.txt',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\Manuales_local\\Oben_chatbot_OpenAI\\transform_manuals\\VOLUME 03 - OperatingManual_clean_output_new.txt',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\Manuales_local\\Oben_chatbot_OpenAI\\transform_manuals\\VOLUME 04 - OperatingManual-Oven_clean_output_new.txt',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\Manuales_local\\Oben_chatbot_OpenAI\\transform_manuals\\VOLUME 05 - OperatingManual-WinderUnit_clean_output_new.txt',
 'c:\\Users\\CarlosGorricho(ext)\\Documents\\Manu

In [36]:
len(txt_docs)

12

In [35]:
type(txt_docs)

list

In [42]:
txt_docs[1].split('.')[0].split('\\')[-1].split('_')[0].strip()

'VOLUME 01 - OperatingManual-LineOverview'

Carga texto de los manuales

In [47]:
clean_text_dict = {}
    
for doc in txt_docs:
    file_name = doc.split('.')[0].split('\\')[-1].split('_')[0].strip()
    print(f'Cargando manual: {file_name}')
    clean_text_dict[file_name] = open(f'{doc}', 'r').read()

Cargando manual: VOLUME 00 - OverviewDocumentation
Cargando manual: VOLUME 01 - OperatingManual-LineOverview
Cargando manual: VOLUME 02 - LineOperationManual
Cargando manual: VOLUME 03 - OperatingManual
Cargando manual: VOLUME 04 - OperatingManual-Oven
Cargando manual: VOLUME 05 - OperatingManual-WinderUnit
Cargando manual: VOLUME 06 - OperatingManual-VacuumSystem
Cargando manual: VOLUME 07 - OperatingManual-InlineCoatingDevice
Cargando manual: VOLUME 08 - OperatingManual-FilmRecycling
Cargando manual: VOLUME 09 - OperatingManual-AuxiliaryWinder
Cargando manual: VOLUME 10 - OperatingManual-DiePreheatingStation
Cargando manual: VOLUME 11 - OperatingManual-PullRollUnit


In [49]:
type(clean_text_dict)

dict

In [50]:
print(clean_text_dict.keys())

dict_keys(['VOLUME 00 - OverviewDocumentation', 'VOLUME 01 - OperatingManual-LineOverview', 'VOLUME 02 - LineOperationManual', 'VOLUME 03 - OperatingManual', 'VOLUME 04 - OperatingManual-Oven', 'VOLUME 05 - OperatingManual-WinderUnit', 'VOLUME 06 - OperatingManual-VacuumSystem', 'VOLUME 07 - OperatingManual-InlineCoatingDevice', 'VOLUME 08 - OperatingManual-FilmRecycling', 'VOLUME 09 - OperatingManual-AuxiliaryWinder', 'VOLUME 10 - OperatingManual-DiePreheatingStation', 'VOLUME 11 - OperatingManual-PullRollUnit'])


Combina texto de todos los archivos

In [51]:
clean_text = ''
largo = 0

for key in clean_text_dict.keys():
    text = clean_text_dict[key]
    print(f'Longitud de {key}: {len(text)}')
    largo += len(text)
    clean_text += text

print(f'{largo}, {len(clean_text)}, {largo == len(clean_text)}')

Longitud de VOLUME 00 - OverviewDocumentation: 1069
Longitud de VOLUME 01 - OperatingManual-LineOverview: 113326
Longitud de VOLUME 02 - LineOperationManual: 370734
Longitud de VOLUME 03 - OperatingManual: 165052
Longitud de VOLUME 04 - OperatingManual-Oven: 80196
Longitud de VOLUME 05 - OperatingManual-WinderUnit: 139024
Longitud de VOLUME 06 - OperatingManual-VacuumSystem: 105061
Longitud de VOLUME 07 - OperatingManual-InlineCoatingDevice: 99324
Longitud de VOLUME 08 - OperatingManual-FilmRecycling: 38021
Longitud de VOLUME 09 - OperatingManual-AuxiliaryWinder: 31817
Longitud de VOLUME 10 - OperatingManual-DiePreheatingStation: 16017
Longitud de VOLUME 11 - OperatingManual-PullRollUnit: 157205
1316846, 1316846, True


In [52]:
chunks = get_text_chunks(clean_text)
len(chunks)

Chunking: procesando texto...
Texto procesado correctamente


793

In [53]:
text_chunks = {}

for key in clean_text_dict.keys():
    print(f'Procesando {key}')
    text_chunks[key] = get_text_chunks(clean_text_dict[key])
    print(f'    {len(text_chunks[key])} chunks')
    

Procesando VOLUME 00 - OverviewDocumentation
Chunking: procesando texto...
Texto procesado correctamente
    1 chunks
Procesando VOLUME 01 - OperatingManual-LineOverview
Chunking: procesando texto...
Texto procesado correctamente
    74 chunks
Procesando VOLUME 02 - LineOperationManual
Chunking: procesando texto...
Texto procesado correctamente
    203 chunks
Procesando VOLUME 03 - OperatingManual
Chunking: procesando texto...
Texto procesado correctamente
    96 chunks
Procesando VOLUME 04 - OperatingManual-Oven
Chunking: procesando texto...
Texto procesado correctamente
    46 chunks
Procesando VOLUME 05 - OperatingManual-WinderUnit
Chunking: procesando texto...
Texto procesado correctamente
    90 chunks
Procesando VOLUME 06 - OperatingManual-VacuumSystem
Chunking: procesando texto...
Texto procesado correctamente
    69 chunks
Procesando VOLUME 07 - OperatingManual-InlineCoatingDevice
Chunking: procesando texto...
Texto procesado correctamente
    61 chunks
Procesando VOLUME 08 - O

In [54]:
list(text_chunks.keys())

['VOLUME 00 - OverviewDocumentation',
 'VOLUME 01 - OperatingManual-LineOverview',
 'VOLUME 02 - LineOperationManual',
 'VOLUME 03 - OperatingManual',
 'VOLUME 04 - OperatingManual-Oven',
 'VOLUME 05 - OperatingManual-WinderUnit',
 'VOLUME 06 - OperatingManual-VacuumSystem',
 'VOLUME 07 - OperatingManual-InlineCoatingDevice',
 'VOLUME 08 - OperatingManual-FilmRecycling',
 'VOLUME 09 - OperatingManual-AuxiliaryWinder',
 'VOLUME 10 - OperatingManual-DiePreheatingStation',
 'VOLUME 11 - OperatingManual-PullRollUnit']

In [56]:
len(text_chunks['VOLUME 01 - OperatingManual-LineOverview'])

74

In [57]:
text_chunks['VOLUME 01 - OperatingManual-LineOverview']

['VOLUME 01 - OperatingManual-LineOverview - Pag 1 \nVOLUME 01 - OperatingManual-LineOverview - Pag 1 \nwOperating manualLine overviewBOPETP.1932 SHB\n\n--------------------------------------------------------------------------------\nVOLUME 01 - OperatingManual-LineOverview - Pag 2 \nVOLUME 01 - OperatingManual-LineOverview - Pag 2 \nw--------------------------------------------------------------------------------\nVOLUME 01 - OperatingManual-LineOverview - Pag 3 \nVOLUME 01 - OperatingManual-LineOverview - Pag 3',
 'w--------------------------------------------------------------------------------\nVOLUME 01 - OperatingManual-LineOverview - Pag 3 \nVOLUME 01 - OperatingManual-LineOverview - Pag 3 \nwImprintThis operating manual is part of the equipment supplied by Brückner MaschinenbauGmbH & Co. KG - hereinafter called Brückner - and describes the necessary pro-cedures for the proper use of the film production line and the associated machines.CopyrightNo part of this operating manual 

### Creación del vector store

In [60]:
len(chunks)

793

In [63]:
get_vector_store(chunks)

Consulta al vector store

In [68]:
global_llm = "gpt-4o"

llm = ChatOpenAI(
    model=global_llm, 
    temperature=0,
    )

In [69]:
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-large'
    )

new_db = Chroma(persist_directory="./chroma_db",
                embedding_function=embeddings,
                )


In [70]:
retriever = new_db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 10, 'fetch_k': 50},
)

In [71]:
multi_retriever = MultiQueryRetriever.from_llm(
    retriever=new_db.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 5, 'fetch_k': 50},
    ), 
    llm=llm
)

In [72]:
template = """Use the following pieces of context to answer the question at the end. Always answer the question in the language in which it is asked. If you don't know the answer, just say that you don't know, don't try to make up an answer. Give your responses primarily in numbered lists. 
    
If the question is related to many documents in the context, try to include the reference for the main titles in your response.

Example:
"user question: procedure for starting the oven?"

"helpful answer: The procedure is detailed in various documents. Here is a summary of the main steps:
1. Main step (Volume 1 - Page 351)
    - Step 1
    - Step 2
2. Secondary step (Volume 10 - Pag 1322)
    - Step 1
    - Step 2"

{context}

Question: {question}

Helpful Answer:
"""

rag_prompt_custom = PromptTemplate.from_template(template)

rag_prompt_custom_chat = ChatPromptTemplate.from_template(template)


In [73]:
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    } 
    | rag_prompt_custom 
    | llm
    | StrOutputParser()
    )

In [74]:
chain_chat = (
        {
        "context": retriever,
        "question": RunnablePassthrough()
    } 
    | rag_prompt_custom_chat
    | llm
    | StrOutputParser()
    )

In [75]:
chain_multi = (
    {
        "context": multi_retriever,
        "question": RunnablePassthrough()
    } 
    | rag_prompt_custom 
    | llm
    | StrOutputParser()
    )

In [76]:
user_question = 'steps to start up a line'

In [77]:
response = chain.invoke(user_question)
print(response)

The steps to start up a film production line are detailed in various documents. Here is a summary of the main steps:

1. **Initial Requirements (Volume 1 - Page 351)**
    - Ensure all safeguards are in place and functioning.
    - Lubrication, inspection, and repair on the line have been carried out.
    - Problems and malfunctions on the film production line have been corrected.
    - The film production line has been cleaned.
    - The main switches, control voltage, control functions, and cooling systems are switched on.
    - All media (e.g., oil, water, steam) are connected and at operating temperature.
    - The raw materials are ready and connected.

2. **Preparations (Volume 1 - Page 351)**
    - Inform all persons on the film production line that the line is being started up.
    - Take measures to ensure the safety of the persons.

3. **Set Start Release (Volume 1 - Page 118)**
    - Ensure the cause for the emergency stop is eliminated.
    - Confirm that no one is standing

In [47]:
response_chat = chain_chat.invoke(user_question)
print(response_chat)

Here are the steps to start an engine based on the provided context:

1. **Preheating the Engine**:
   - Operate the cooling water heating system.
   - Circulate heating water through the cylinders for at least 2 hours before starting.
   - Ensure the temperature of the water is over 60°C for HFO and over 40°C for MDO.

2. **Preparation of Fuel System**:
   - Drain fuel tanks for water and check filters.
   - Check fuel tank level and fill if needed.
   - Check the cooling system for diesel fuel oil.
   - For starting on HFO, recheck preheating of the engine ready for HFO circulation.

3. **Starting Sequence**:
   - Ensure all start requests are present at the end of the Start Delay timer.
   - Energize the fuel relay and crank the engine.
   - If the engine fails to fire, disengage the starter motor for the Crank Rest duration and attempt to start again.
   - If the engine fires, disengage the starter motor.
   - Activate the Safety On Delay timer to allow fault inputs to stabilize.



In [78]:
docs = retriever.invoke(user_question)
docs

[Document(page_content='l6 Starting up the film production line6.1 Requirement\n\nFaulty or ineffective safeguards.The protective function is lost.Life-threatening situations or very serious injuries may occur.►Put the safeguards back in place after the maintenance work is complete.►Check that the safeguards are operating correctly and effectively.►Do not bypass safeguards.►Do not render safeguards ineffective.►Do not change the arrangement of the safeguards.The following film production line requirements must be met:• Lubrication, inspection and repair on the line have been carried out.• Problems and malfunctions on the film production line have been corrected.Malfunctions on the film production line have been reset.• The film production line has been cleaned.• The film production line is ready for operation:The main switches are switched on.The control voltage or the uninterruptible power supply is switched on.The control functions are switched on.The cooling systems are switched on.

In [79]:
response_multi = chain_multi.invoke(user_question)
print(response_multi)

The steps to start up a film production line are detailed in various documents. Here is a summary of the main steps:

1. **Initial Requirements (Volume 1 - Page 351)**
    - Ensure all safeguards are in place and functioning.
    - Lubrication, inspection, and repair on the line have been carried out.
    - Problems and malfunctions on the film production line have been corrected.
    - The film production line has been cleaned.
    - The main switches, control voltage, control functions, and cooling systems are switched on.
    - All media (e.g., oil, water, steam) are connected and at operating temperature.
    - The raw materials are ready and connected.

2. **Preparations (Volume 1 - Page 351)**
    - Inform all persons on the film production line that the line is being started up.
    - Take measures to ensure the safety of the persons.

3. **Set Start Release (Volume 1 - Page 104)**
    - Ensure the cause for the emergency stop is eliminated.
    - Close the service hatches.
    

In [80]:
docs_multi = multi_retriever.invoke(user_question)
docs_multi

[Document(page_content='l6 Starting up the film production line6.1 Requirement\n\nFaulty or ineffective safeguards.The protective function is lost.Life-threatening situations or very serious injuries may occur.►Put the safeguards back in place after the maintenance work is complete.►Check that the safeguards are operating correctly and effectively.►Do not bypass safeguards.►Do not render safeguards ineffective.►Do not change the arrangement of the safeguards.The following film production line requirements must be met:• Lubrication, inspection and repair on the line have been carried out.• Problems and malfunctions on the film production line have been corrected.Malfunctions on the film production line have been reset.• The film production line has been cleaned.• The film production line is ready for operation:The main switches are switched on.The control voltage or the uninterruptible power supply is switched on.The control functions are switched on.The cooling systems are switched on.

In [82]:
for doc in docs_multi:
    print(len(doc.page_content))

2032
627
1146
2029
1760
1997
592
1344
1195
613
2187
2038
2184
1379
1725


In [83]:
user_question = 'cuales son los pasos para poner en marcha la linea?'


In [84]:
response = chain.invoke(user_question)
print(response)

Los pasos para poner en marcha la línea están detallados en varios documentos. Aquí hay un resumen de los pasos principales:

1. **Comprobación de salvaguardas y preparación inicial** (VOLUME 02 - LineOperationManual - Pag 36-37)
    - Verificar que todas las salvaguardas estén montadas y funcionando.
    - Realizar una inspección visual de la línea de producción de películas.
    - Comprobar la preparación operativa.
    - Configurar el sistema de suministro de resina si es necesario.
    - Encender el sistema de extracción de la unidad de matriz de fusión y verificar su funcionamiento.
    - Calentar el sistema de extrusión.

2. **Comprobación de la unidad de bobinado** (VOLUME 02 - LineOperationManual - Pag 207-208)
    - Encender el eje de bobinado y el soporte del rodillo de contacto.
    - Verificar que no haya enclavamientos activos en los accionamientos.

3. **Comprobación de la unidad hidráulica** (VOLUME 02 - LineOperationManual - Pag 208)
    - Verificar las unidades de mant

In [85]:
response_chat = chain_chat.invoke(user_question)
print(response_chat)

Los pasos para poner en marcha la línea están detallados en varios documentos. Aquí hay un resumen de los pasos principales:

1. **Comprobación de salvaguardas y preparación inicial** (VOLUME 02 - LineOperationManual - Pag 36)
    - Verificar que todas las salvaguardas estén montadas y funcionando.
    - Realizar una inspección visual de la línea de producción de películas.
    - Comprobar la preparación operativa (mensajes de advertencia, estaciones de operación, señales visuales y acústicas).

2. **Preparación del sistema de resina y extrusión** (VOLUME 02 - LineOperationManual - Pag 36-37)
    - Configurar el sistema de suministro de resina si es necesario.
    - Encender el sistema de escape de la unidad de matriz de fusión y verificar su funcionamiento.
    - Calentar el sistema de extrusión.

3. **Comprobación y puesta en marcha de componentes específicos** (VOLUME 02 - LineOperationManual - Pag 207-208)
    - Encender la unidad de bobinado (eje de bobinado y soporte de rodillo d

In [86]:
docs = retriever.invoke(user_question)
docs

[Document(page_content='VOLUME 01 - OperatingManual-LineOverview - Pag 154 \nVOLUME 01 - OperatingManual-LineOverview - Pag 154 \nwCommissioning aftermalfunction\n\nAfter all faults and triggered safeguards have been reset, the line components canbe started again.The initial commissioning after a malfunction must be carried out under the constantobservation of an instructed specialist.\n\n--------------------------------------------------------------------------------\nVOLUME 01 - OperatingManual-LineOverview - Pag 155 \nVOLUME 01 - OperatingManual-LineOverview - Pag 155'),
 Document(page_content='VOLUME 02 - LineOperationManual - Pag 36 \nVOLUME 02 - LineOperationManual - Pag 36 \nl►Check the safeguards.• All safeguards are mounted and functioning.►Carry out a visual inspection of the film production line.• The infrastructure is not damaged or faulty.• The transportation routes are level and not blocked.• The film production line is sufficiently illuminated.• The components are not da

In [87]:
response_multi = chain_multi.invoke(user_question)
print(response_multi)

Los pasos para poner en marcha la línea de producción de películas están detallados en varios documentos. Aquí hay un resumen de los pasos principales:

1. **Requisitos previos (Document - Page 16)**
    - Lubricación, inspección y reparación de la línea.
    - Corrección de problemas y malfunciones.
    - Limpieza de la línea.
    - Conexión y cumplimiento de todos los medios (aceite térmico, agua, aire comprimido, vapor).
    - Preparación de las materias primas.

2. **Preparaciones (Document - Page 16)**
    - Informar a todas las personas en la línea de producción.
    - Tomar medidas para garantizar la seguridad de las personas.

3. **Comprobaciones iniciales (Document - Page 36)**
    - Verificar que todas las salvaguardas estén montadas y funcionando.
    - Inspección visual de la línea de producción.
    - Comprobar la preparación operativa (mensajes de advertencia, estaciones de operación, señales visuales y acústicas).

4. **Ajustes y configuraciones (Document - Page 160)**
 

In [88]:
docs_multi = multi_retriever.invoke(user_question)
docs_multi

[Document(page_content='l6 Starting up the film production line6.1 Requirement\n\nFaulty or ineffective safeguards.The protective function is lost.Life-threatening situations or very serious injuries may occur.►Put the safeguards back in place after the maintenance work is complete.►Check that the safeguards are operating correctly and effectively.►Do not bypass safeguards.►Do not render safeguards ineffective.►Do not change the arrangement of the safeguards.The following film production line requirements must be met:• Lubrication, inspection and repair on the line have been carried out.• Problems and malfunctions on the film production line have been corrected.Malfunctions on the film production line have been reset.• The film production line has been cleaned.• The film production line is ready for operation:The main switches are switched on.The control voltage or the uninterruptible power supply is switched on.The control functions are switched on.The cooling systems are switched on.

### Crea consulta con fuente de documentos

In [89]:
from langchain_core.runnables import RunnableParallel

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | rag_prompt_custom
    | llm
    | StrOutputParser()
    )

rag_chain_with_source = RunnableParallel(
    {"context": multi_retriever, "question": RunnablePassthrough()}
    ).assign(answer=rag_chain_from_docs)

response_source = rag_chain_with_source.invoke(user_question)

In [90]:
response_source

{'context': [Document(page_content='l6 Starting up the film production line6.1 Requirement\n\nFaulty or ineffective safeguards.The protective function is lost.Life-threatening situations or very serious injuries may occur.►Put the safeguards back in place after the maintenance work is complete.►Check that the safeguards are operating correctly and effectively.►Do not bypass safeguards.►Do not render safeguards ineffective.►Do not change the arrangement of the safeguards.The following film production line requirements must be met:• Lubrication, inspection and repair on the line have been carried out.• Problems and malfunctions on the film production line have been corrected.Malfunctions on the film production line have been reset.• The film production line has been cleaned.• The film production line is ready for operation:The main switches are switched on.The control voltage or the uninterruptible power supply is switched on.The control functions are switched on.The cooling systems are 

In [91]:
response_source['question']

'cuales son los pasos para poner en marcha la linea?'

In [92]:
response_source['answer']

'Los pasos para poner en marcha la línea de producción de películas están detallados en varios documentos. Aquí hay un resumen de los pasos principales:\n\n1. **Requisitos previos** (VOLUME 03 - Operating Manual - Pag 13)\n    - Lubricación, inspección y reparación de la línea completadas.\n    - Problemas y malfunciones corregidos.\n    - La línea de producción ha sido limpiada.\n    - Los interruptores principales están encendidos.\n    - El voltaje de control o la fuente de alimentación ininterrumpida está encendida.\n    - Los sistemas de enfriamiento están encendidos.\n    - Todos los medios están conectados y cumplen con los requisitos.\n    - Las materias primas están listas.\n\n2. **Preparaciones** (VOLUME 03 - Operating Manual - Pag 13)\n    - Informar a todas las personas en la línea de producción que la línea se está poniendo en marcha.\n    - Tomar medidas para garantizar la seguridad de las personas.\n\n3. **Actividades generales** (VOLUME 02 - Line Operation Manual - Pag 

In [93]:
response_source['context']

[Document(page_content='l6 Starting up the film production line6.1 Requirement\n\nFaulty or ineffective safeguards.The protective function is lost.Life-threatening situations or very serious injuries may occur.►Put the safeguards back in place after the maintenance work is complete.►Check that the safeguards are operating correctly and effectively.►Do not bypass safeguards.►Do not render safeguards ineffective.►Do not change the arrangement of the safeguards.The following film production line requirements must be met:• Lubrication, inspection and repair on the line have been carried out.• Problems and malfunctions on the film production line have been corrected.Malfunctions on the film production line have been reset.• The film production line has been cleaned.• The film production line is ready for operation:The main switches are switched on.The control voltage or the uninterruptible power supply is switched on.The control functions are switched on.The cooling systems are switched on.

In [95]:
all_pages = []

for doc in response_source['context']:
    print('**'*50)
    page = ((re.findall('(VOLUME\s+\d+\s+-\s+([A-Za-z\-])*\s+-\s+Pag\s+\d+\s+)', doc.page_content))) 
    print(doc.page_content)
    print(page)
    all_pages.extend(page)

source_pages = '\n'.join([pg for pg in sorted(set(all_pages))])

print(source_pages)


****************************************************************************************************
l6 Starting up the film production line6.1 Requirement

Faulty or ineffective safeguards.The protective function is lost.Life-threatening situations or very serious injuries may occur.►Put the safeguards back in place after the maintenance work is complete.►Check that the safeguards are operating correctly and effectively.►Do not bypass safeguards.►Do not render safeguards ineffective.►Do not change the arrangement of the safeguards.The following film production line requirements must be met:• Lubrication, inspection and repair on the line have been carried out.• Problems and malfunctions on the film production line have been corrected.Malfunctions on the film production line have been reset.• The film production line has been cleaned.• The film production line is ready for operation:The main switches are switched on.The control voltage or the uninterruptible power supply is switched on

TypeError: sequence item 0: expected str instance, tuple found

In [97]:
response_source['context']

[Document(page_content='l6 Starting up the film production line6.1 Requirement\n\nFaulty or ineffective safeguards.The protective function is lost.Life-threatening situations or very serious injuries may occur.►Put the safeguards back in place after the maintenance work is complete.►Check that the safeguards are operating correctly and effectively.►Do not bypass safeguards.►Do not render safeguards ineffective.►Do not change the arrangement of the safeguards.The following film production line requirements must be met:• Lubrication, inspection and repair on the line have been carried out.• Problems and malfunctions on the film production line have been corrected.Malfunctions on the film production line have been reset.• The film production line has been cleaned.• The film production line is ready for operation:The main switches are switched on.The control voltage or the uninterruptible power supply is switched on.The control functions are switched on.The cooling systems are switched on.

In [106]:
page = response_source['context'][10].page_content
page

'VOLUME 02 - LineOperationManual - Pag 182 \nVOLUME 02 - LineOperationManual - Pag 182 \nl16.3 Switching the transverse direction orienter off and on16.3.1 Switching on the transverse direction orienterRequirement• No interlocks on the drives are active.The push button on the operator panel for switching on the drives flashes.Required personnel2 operators are required.• 1 operator at the operator panel that is located at the transverse direction ori-enter inlet.• 1 operator at the operator panel that is located at the transverse direction ori-enter outlet.Safety WARNINGMoving machine parts can crush body parts.This can lead to serious injury.►Do not reach into the vicinity of moving machine parts.►Wait for the machine to come to a standstill.►Before actuating the machine parts, ensure that no one is located in the dangerzone.\n\nDuring operation, small parts, dust or chips can fly out at the outlet of thetransverse direction orienter.This can cause injury to the eyes.►Do not enter the 

In [108]:
all_pages

[('VOLUME 03 - OperatingManual - Pag 13 \n', 'l'),
 ('VOLUME 03 - OperatingManual - Pag 13 \n', 'l'),
 ('VOLUME 03 - OperatingManual - Pag 14 \n', 'l'),
 ('VOLUME 03 - OperatingManual - Pag 14 \n', 'l'),
 ('VOLUME 03 - OperatingManual - Pag 15 \n', 'l'),
 ('VOLUME 01 - OperatingManual-LineOverview - Pag 104 \n', 'w'),
 ('VOLUME 01 - OperatingManual-LineOverview - Pag 104 \n', 'w'),
 ('VOLUME 01 - OperatingManual-LineOverview - Pag 105 \n', 'w'),
 ('VOLUME 03 - OperatingManual - Pag 59 \n', 'l'),
 ('VOLUME 03 - OperatingManual - Pag 59 \n', 'l'),
 ('VOLUME 03 - OperatingManual - Pag 60 \n', 'l'),
 ('VOLUME 03 - OperatingManual - Pag 60 \n', 'l'),
 ('VOLUME 03 - OperatingManual - Pag 61 \n', 'l'),
 ('VOLUME 02 - LineOperationManual - Pag 160 \n', 'l'),
 ('VOLUME 02 - LineOperationManual - Pag 160 \n', 'l'),
 ('VOLUME 02 - LineOperationManual - Pag 161 \n', 'l'),
 ('VOLUME 01 - OperatingManual-LineOverview - Pag 154 \n', 'w'),
 ('VOLUME 01 - OperatingManual-LineOverview - Pag 154 \n', 'w'

In [109]:
re.findall(r'(VOLUME\s+\d+\s+-\s+[A-Za-z\-]*\s+-\s+Pag\s+\d+\s+)', page)

['VOLUME 02 - LineOperationManual - Pag 182 \n',
 'VOLUME 02 - LineOperationManual - Pag 182 \n',
 'VOLUME 02 - LineOperationManual - Pag 183 \n']

In [168]:
response_string = f"{response_source['answer']}\n\nFuente (Source)\n\n{source_pages}"
print(response_string)

Los pasos para poner en marcha un motor pueden variar dependiendo del tipo de motor y del sistema específico. A continuación, se presentan algunos pasos generales basados en el contexto proporcionado:

1. **Precalentamiento del Motor**:
   - Operar el sistema de calentamiento del agua de enfriamiento.
   - Asegurarse de que el agua de calentamiento circule a través de los cilindros durante al menos 2 horas antes de arrancar.
   - La temperatura del agua debe ser adecuada según el tipo de combustible (HFO: >60°C, MDO: >40°C).

2. **Preparación del Sistema de Combustible**:
   - Drenar los tanques de combustible para eliminar agua y verificar los filtros.
   - Comprobar el nivel del tanque de combustible y rellenar si es necesario.
   - Verificar el sistema de enfriamiento para el aceite diesel.
   - Para arranques en HFO, asegurarse de que el motor esté precalentado y listo para la circulación de HFO.

3. **Secuencia de Arranque**:
   - Si el motor está configurado para CAN, las ECU com

In [155]:
response_string

'1. **Preparaciones Previas:**\n   - Verificar el nivel de combustible, nivel de refrigerante y nivel de aceite del motor.\n   - Asegurarse de que el nivel de aceite no supere la marca máxima en la varilla de medición.\n   - Utilizar combustibles, lubricantes y refrigerantes aprobados.\n\n2. **Inicio del Motor:**\n   - Insertar la llave en el interruptor de arranque.\n   - Mover la palanca de control a "Velocidad de ralentí".\n   - Girar el interruptor de llave en el sentido de las agujas del reloj.\n   - No operar el motor de arranque por más de 10 segundos a la vez.\n   - Después de la ignición del motor, soltar la llave.\n   - Ajustar la palanca de control a la velocidad deseada.\n   - Si el motor no arranca, soltar la llave, esperar aproximadamente 1 minuto y luego intentar nuevamente.\n\n3. **Verificación Durante el Arranque:**\n   - Comprobar la presión del aceite del motor.\n   - Verificar la temperatura del agua.\n   - Revisar el medidor de voltaje de la batería.\n   - Observar

In [156]:
from IPython.display import display, Markdown, Latex
display(Markdown(response_string))

1. **Preparaciones Previas:**
   - Verificar el nivel de combustible, nivel de refrigerante y nivel de aceite del motor.
   - Asegurarse de que el nivel de aceite no supere la marca máxima en la varilla de medición.
   - Utilizar combustibles, lubricantes y refrigerantes aprobados.

2. **Inicio del Motor:**
   - Insertar la llave en el interruptor de arranque.
   - Mover la palanca de control a "Velocidad de ralentí".
   - Girar el interruptor de llave en el sentido de las agujas del reloj.
   - No operar el motor de arranque por más de 10 segundos a la vez.
   - Después de la ignición del motor, soltar la llave.
   - Ajustar la palanca de control a la velocidad deseada.
   - Si el motor no arranca, soltar la llave, esperar aproximadamente 1 minuto y luego intentar nuevamente.

3. **Verificación Durante el Arranque:**
   - Comprobar la presión del aceite del motor.
   - Verificar la temperatura del agua.
   - Revisar el medidor de voltaje de la batería.
   - Observar el medidor de presión.

4. **Calentamiento del Motor:**
   - Calentar el motor durante 5 a 10 minutos.
   - Durante el calentamiento, verificar la presión del aceite, la temperatura del agua y el voltaje de la batería.
   - Escuchar sonidos y vibraciones anormales. Si se detectan, detener el motor presionando el botón de parada de emergencia.

5. **Operación Normal:**
   - Asegurarse de que el motor funcione sin problemas y sin ruidos o vibraciones anormales.
   - Monitorear continuamente los indicadores de presión de aceite, temperatura del agua y voltaje de la batería.

6. **Parada del Motor:**
   - Para detener el motor, presionar el botón de parada de emergencia si es necesario.

Estos pasos aseguran un arranque seguro y eficiente del motor, minimizando el riesgo de daños y garantizando un funcionamiento óptimo.

Fuente (Source)

VOLUME 10 - Pag 1515 

VOLUME 10 - Pag 1527 
VOLUME 10 - Pag 1528 

VOLUME 10 - Pag 1776 

VOLUME 10 - Pag 437 
VOLUME 10 - Pag 558 

VOLUME 2 - Pag 584 

VOLUME 6 - Pag 332 

VOLUME 7 - Pag 54 

VOLUME 7 - Pag 55 

VOLUME 8 - Pag 1093 


In [ ]:
response_source['question']

In [ ]:
print(response_source['answer'])